In [1]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [2]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [4]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:4000,
    GenParams.MIN_NEW_TOKENS:1,
    # "temperature":0.1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```","[/INST]","<|user|>","<|endoftext|>","<|assistant|>"],
}

model = ModelInference(model_id="ibm/granite-3-8b-instruct", credentials=creds, params=params, project_id=project_id)

In [5]:
model_id = GRANITE_3_8B_INSTRUCT
language = "javascript"
documentframework = "jsdoc"
testframework = "mocha"

In [6]:
prompt = f"""<|user|>
Please generate a sample {language} code:
-give more detail and complex logic.
-generate the code only.
-dont generate explanation.
<|assistant|>
```{language}
"""

sample_code = model.generate_text(prompt).replace("```","")
print(sample_code)

const users = [
  { id: 1, name: 'John', email: 'john@example.com' },
  { id: 2, name: 'Jane', email: 'jane@example.com' },
  { id: 3, name: 'Doe', email: 'doe@example.com' },
];

const validateEmail = (email) => {
  const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return regex.test(email);
};

const findUserById = (id) => {
  return users.find(user => user.id === id);
};

const findUserByEmail = (email) => {
  return users.find(user => user.email === email);
};

const addUser = (user) => {
  users.push(user);
};

const removeUser = (id) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users.splice(index, 1);
  }
};

const updateUser = (id, user) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users[index] = user;
  }
};

const searchUsers = (query) => {
  return users.filter(user => user.name.includes(query) || user.email.includes(query));
};

const sortUsers = (sortBy, order) => {
  return users.sort((a, b) =

In [9]:
sample_code = """const users = [
  { id: 1, name: 'John', email: 'john@example.com' },
  { id: 2, name: 'Jane', email: 'jane@example.com' },
  { id: 3, name: 'Doe', email: 'doe@example.com' },
];

const validateEmail = (email) => {
  const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return regex.test(email);
};

const findUserById = (id) => {
  return users.find(user => user.id === id);
};

const findUserByEmail = (email) => {
  return users.find(user => user.email === email);
};

const addUser = (user) => {
  users.push(user);
};

const removeUser = (id) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users.splice(index, 1);
  }
};

const updateUser = (id, user) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users[index] = user;
  }
};

const searchUsers = (query) => {
  return users.filter(user => user.name.includes(query) || user.email.includes(query));
};

const sortUsers = (sortBy, order) => {
  return users.sort((a, b) => {
    if (order === 'asc') {
      return a[sortBy] < b[sortBy] ? -1 : a[sortBy] > b[sortBy] ? 1 : 0;
    } else {
      return a[sortBy] > b[sortBy] ? -1 : a[sortBy] < b[sortBy] ? 1 : 0;
    }
  });
};

const paginateUsers = (page, pageSize) => {
  const start = (page - 1) * pageSize;
  return users.slice(start, start + pageSize);
};

const exportUsers = () => {
  return JSON.stringify(users);
};

const importUsers = (usersData) => {
  users.length = 0;
  users.push(...usersData);
};
"""

bad_code = """const users = [
  { id: 1, name: 'John', email: 'john@example.com' },
  { id: 2, name: 'Jane', email: 'jane@example.com' },
  { id: 3, name: 'Doe', email: 'doe@example.com' },
];

const validateEmail = (email) => {
  const regex = /^[^\s@]+@[^\s@
};

const findUserById = (id) => {
  return users.find(user => user.id === id);
};

const findUserByEmail = (email) => {
  return users.find(user => user.email === email);
};

const addUser = (user) => {
  users.push(user);
};

const removeUser = (id) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users.splice(index, 1);
  }
};

const updateUser = (id, user) => {
  const index = users.find
    users[index] = user;
  }
};

const searchUsers = (query) => {
  return users.filter(user => useery) || user.email.includes(query));
};

const sortUsers = (sortBy, order) => {
  return users.sort((a, b) => {
    if (order === 'asc') {
      return a[sortBy] < b[sortBy] ? -1 : a[sortBy] > b[sortBy] ? 1 : 0;
    } else {
      return a[sortBy] > b[s
  });
};

const paginateUsers = (page, pageSize) => {
  const start = (page - 1) * pageSize;
  return users.slice(start, start + pageSize);
};

const exportUsers = () => {
  return JSON.stringify(users);
};

const importUsers = (usersData) => {
  users.length = 0;
  users.push(...usersData);
};

"""

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:67: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:67: SyntaxWarning: invalid escape sequence '\s'
/var/folders/nr/1z5zq4yj1cjgn45lr8fydpc00000gn/T/ipykernel_52771/4038921736.py:1: SyntaxWarning: invalid escape sequence '\s'
  sample_code = """const users = [
/var/folders/nr/1z5zq4yj1cjgn45lr8fydpc00000gn/T/ipykernel_52771/4038921736.py:67: SyntaxWarning: invalid escape sequence '\s'
  bad_code = """const users = [


gnerate explanation

In [11]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.
-dont generate code.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = model.generate_text(prompt)
print(answer)


This JavaScript code defines a module for managing a list of users. Here's a breakdown of the functions:

1. `users`: An array of objects, each representing a user with properties `id`, `name`, and `email`.

2. `validateEmail(email)`: A function that checks if an email is valid using a regular expression. It returns `true` if the email is valid and `false` otherwise.

3. `findUserById(id)`: A function that finds a user by their ID and returns the user object if found, or `undefined` if not found.

4. `findUserByEmail(email)`: A function that finds a user by their email and returns the user object if found, or `undefined` if not found.

5. `addUser(user)`: A function that adds a new user to the `users` array.

6. `removeUser(id)`: A function that removes a user from the `users` array by their ID.

7. `updateUser(id, user)`: A function that updates an existing user in the `users` array by their ID.

8. `searchUsers(query)`: A function that searches the `users` array for users whose name

generate jsdoc

In [12]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


/**
 * An array of user objects.
 * @type {Array<Object>}
 */
const users = [
  { id: 1, name: 'John', email: 'john@example.com' },
  { id: 2, name: 'Jane', email: 'jane@example.com' },
  { id: 3, name: 'Doe', email: 'doe@example.com' },
];

/**
 * Validates an email address.
 * @param {string} email - The email address to validate.
 * @returns {boolean} - True if the email is valid, false otherwise.
 */
const validateEmail = (email) => {
  const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return regex.test(email);
};

/**
 * Finds a user by their ID.
 * @param {number} id - The ID of the user to find.
 * @returns {Object} - The user object if found, null otherwise.
 */
const findUserById = (id) => {
  return users.find(user => user.id === id);
};

/**
 * Finds a user by their email address.
 * @param {string} email - The email address of the user to find.
 * @returns {Object} - The user object if found, null otherwise.
 */
const findUserByEmail = (email) => {
  return users.find(user => 

score, review and improve

In [14]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-make it more object oriented.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
-dont generate code.
<|assistant|>
score and review:"""

suggestion = model.generate_text(prompt)
print(suggestion)



The provided JavaScript code is a well-structured and functional implementation of a user management system. It includes various methods for adding, removing, updating, searching, sorting, paginating, and exporting/importing users. The code is clean, readable, and follows good practices.

However, there are a few areas that could be improved to make the code more object-oriented and maintainable. Here are my suggestions:

1. Encapsulation: The `users` array is a global variable, which can lead to issues with scope and potential conflicts with other parts of the codebase. Consider encapsulating this data within a class or module.

2. Validation: The `validateEmail` function is a good start, but it could be expanded to include more comprehensive email validation, such as checking for the presence of a domain and top-level domain.

3. Error Handling: The functions `findUserById`, `findUserByEmail`, `removeUser`, and `updateUser` could benefit from error handling. For example, they could

In [15]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
-dont generate explanation.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


class UserManager {
  constructor() {
    this.users = [
      { id: 1, name: 'John', email: 'john@example.com' },
      { id: 2, name: 'Jane', email: 'jane@example.com' },
      { id: 3, name: 'Doe', email: 'doe@example.com' },
    ];
  }

  validateEmail(email) {
    const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    return regex.test(email);
  }

  findUserById(id) {
    const user = this.users.find(user => user.id === id);
    if (!user) {
      throw new Error(`User with ID ${id} not found.`);
    }
    return user;
  }

  findUserByEmail(email) {
    const user = this.users.find(user => user.email === email);
    if (!user) {
      throw new Error(`User with email ${email} not found.`);
    }
    return user;
  }

  addUser(user) {
    if (!user || typeof user !== 'object' || !user.id || !user.name || !user.email) {
      throw new Error('Invalid user object.');
    }
    this.users.push(user);
  }

  removeUser(id) {
    const index = this.users.findIndex(user => user.id === id);


fix bad code

In [16]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


const users = [
  { id: 1, name: 'John', email: 'john@example.com' },
  { id: 2, name: 'Jane', email: 'jane@example.com' },
  { id: 3, name: 'Doe', email: 'doe@example.com' },
];

const validateEmail = (email) => {
  // The regular expression for email validation is incomplete.
  // It should include the entire email format, including the '@' symbol and domain.
  const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return regex.test(email);
};

const findUserById = (id) => {
  return users.find(user => user.id === id);
};

const findUserByEmail = (email) => {
  return users.find(user => user.email === email);
};

const addUser = (user) => {
  users.push(user);
};

const removeUser = (id) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users.splice(index, 1);
  }
};

const updateUser = (id, user) => {
  const index = users.findIndex(user => user.id === id);
  if (index !== -1) {
    users[index] = user; // Update the user at the found index
  }
};

cons

generate diagram

In [17]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


graph TD
A[const users] --> B[User 1]
A --> C[User 2]
A --> D[User 3]
B --> E[id: 1]
B --> F[name: John]
B --> G[email: john@example.com]
C --> H[id: 2]
C --> I[name: Jane]
C --> J[email: jane@example.com]
D --> K[id: 3]
D --> L[name: Doe]
D --> M[email: doe@example.com]
N[const validateEmail] --> O[regex]
O --> P[test]
N --> Q[email]
Q --> R[findUserById]
R --> S[id]
Q --> T[findUserByEmail]
T --> U[email]
Q --> V[addUser]
V --> W[user]
Q --> X[removeUser]
X --> Y[id]
Q --> Z[updateUser]
Z --> AA[id]
Z --> AB[user]
Q --> AC[searchUsers]
AC --> AD[query]
Q --> AE[sortUsers]
AE --> AF[sortBy]
AE --> AG[order]
Q --> AH[paginateUsers]
AH --> AI[page]
AH --> AJ[pageSize]
Q --> AK[exportUsers]
Q --> AL[importUsers]
AL --> AM[usersData]



conversion

In [21]:
target_language = "angular"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import { Component } from '@angular/core';

@Component({
  selector: 'app-root',
  templateUrl: './app.component.html',
  styleUrls: ['./app.component.css']
})
export class AppComponent {
  users = [
    { id: 1, name: 'John', email: 'john@example.com' },
    { id: 2, name: 'Jane', email: 'jane@example.com' },
    { id: 3, name: 'Doe', email: 'doe@example.com' },
  ];

  validateEmail(email: string): boolean {
    const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    return regex.test(email);
  }

  findUserById(id: number): any {
    return this.users.find(user => user.id === id);
  }

  findUserByEmail(email: string): any {
    return this.users.find(user => user.email === email);
  }

  addUser(user: any): void {
    this.users.push(user);
  }

  removeUser(id: number): void {
    const index = this.users.findIndex(user => user.id === id);
    if (index !== -1) {
      this.users.splice(index, 1);
    }
  }

  updateUser(id: number, user: any): void {
    const index = this.users.findInd

generate api definition

In [19]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
-dont generate explanation.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = model.generate_text(prompt).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: User Management API
  version: 1.0.0
paths:
  /users:
    get:
      summary: Get users
      parameters:
        - name: query
          in: query
          description: Search query
          schema:
            type: string
        - name: sortBy
          in: query
          description: Sort by field
          schema:
            type: string
        - name: order
          in: query
          description: Sort order (asc or desc)
          schema:
            type: string
        - name: page
          in: query
          description: Page number
          schema:
            type: integer
        - name: pageSize
          in: query
          description: Page size
          schema:
            type: integer
      responses:
        '200':
          description: OK
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/User'
    post:
   

generate test case

In [20]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
-generate the test only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


const { expect } = require('chai');
const { addUser, removeUser, updateUser, searchUsers, sortUsers, paginateUsers, exportUsers, importUsers } = require('./userManagement');

describe('User Management', () => {
  let users;

  beforeEach(() => {
    users = [
      { id: 1, name: 'John', email: 'john@example.com' },
      { id: 2, name: 'Jane', email: 'jane@example.com' },
      { id: 3, name: 'Doe', email: 'doe@example.com' },
    ];
  });

  describe('addUser', () => {
    it('should add a new user', () => {
      const newUser = { id: 4, name: 'New User', email: 'newuser@example.com' };
      addUser(newUser);
      expect(users).to.include(newUser);
    });

    it('should not add a user with an invalid email', () => {
      const invalidUser = { id: 5, name: 'Invalid User', email: 'invalid' };
      expect(() => addUser(invalidUser)).to.throw('Invalid email');
    });
  });

  describe('removeUser', () => {
    it('should remove a user by id', () => {
      removeUser(2);
      e